In [ ]:
// using nightly-build
//#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22356.1"
#r "nuget: Microsoft.ML, 2.0.0-preview.22356.1"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"
#r "nuget:SandDance.InteractiveExtension, 1.0.53"
#r "nuget: XPlot.Plotly.Interactive, 4.0.6"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json

Error: C:\Users\rossl\.packagemanagement\nuget\Projects\21444--1158b7cd-86d4-419d-8e97-80f7c61288d8\Project.fsproj : warning NU1608: Detected package version outside of dependency constraint: Microsoft.DotNet.Interactive.PowerShell 1.0.0-beta.21559.2 requires Microsoft.DotNet.Interactive (= 1.0.0-beta.21559.2) but version Microsoft.DotNet.Interactive 1.0.0-beta.22371.2 was resolved.
C:\Users\rossl\.packagemanagement\nuget\Projects\21444--1158b7cd-86d4-419d-8e97-80f7c61288d8\Project.fsproj : warning NU1608: Detected package version outside of dependency constraint: Microsoft.DotNet.Interactive.Http 1.0.0-beta.21559.2 requires Microsoft.DotNet.Interactive (= 1.0.0-beta.21559.2) but version Microsoft.DotNet.Interactive 1.0.0-beta.22371.2 was resolved.
C:\Users\rossl\.packagemanagement\nuget\Projects\21444--1158b7cd-86d4-419d-8e97-80f7c61288d8\Project.fsproj : error NU1107: Version conflict detected for Microsoft.DotNet.Interactive.Formatting. Install/reference Microsoft.DotNet.Interacti

# Exploring your own data - NFL Draft history from 2015 to 2021

OK, Let's do some using statements so that we have tools to work with our data.

In [ ]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.DotNet.Interactive;
using XPlot.Plotly.Interactive;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using System.Collections.Generic;
using System.IO;

Do our files exist? They were downloaded from [pro-football-reference.com](https://www.pro-football-reference.com/), such as this page for the [2015 Draft](https://www.pro-football-reference.com/years/2015/draft.htm).

In [ ]:
var fileExists = File.Exists("drafts/2015Draft.csv");
if (fileExists)
{
    display("This file exists.  Nothing has blown up.....yet.");
}

OK.  Make separate data frame objects for each draft from 2015 to 2021. Also, show the 2015 draft, to make sure the data makes sense.

In [ ]:
var draft2015 = DataFrame.LoadCsv("drafts/2015Draft.csv");
var draft2016 = DataFrame.LoadCsv("drafts/2016Draft.csv");
var draft2017 = DataFrame.LoadCsv("drafts/2017Draft.csv");
var draft2018 = DataFrame.LoadCsv("drafts/2018Draft.csv");
var draft2019 = DataFrame.LoadCsv("drafts/2019Draft.csv");
var draft2020 = DataFrame.LoadCsv("drafts/2020Draft.csv");
var draft2021 = DataFrame.LoadCsv("drafts/2021Draft.csv");
draft2015.Sample(15)

Let's try to look at this data in SandDance.

In [ ]:
draft2021.ToTabularDataResource().ExploreWithSandDance().Display()

I don't think that worked.  It's because the dataFrame has null values.

- For "To", let's assume the year they were drafted was their last year. That means we need a DraftYear column.
- For null age, let's guess the median age of all the other draftees, rounded down.
- "0" values for all other columns seem fine.

First, Add the DraftYear and make it the year of that draft.

In [ ]:
PrimitiveDataFrameColumn<int> draftYear = new PrimitiveDataFrameColumn<int>("Ints", 256); // Makes a column of length 3. Filled with nulls initially

draft2021.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2021.Rows.Count()));
draft2021["DraftYear"].FillNulls(2021, true);
draft2021["To"].FillNulls(2021, true);
draft2020.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2020.Rows.Count()));
draft2020["DraftYear"].FillNulls(2020, true);
draft2020["To"].FillNulls(2020, true);
draft2019.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2019.Rows.Count()));
draft2019["DraftYear"].FillNulls(2019, true);
draft2019["To"].FillNulls(2019, true);
draft2018.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2018.Rows.Count()));
draft2018["DraftYear"].FillNulls(2018, true);
draft2018["To"].FillNulls(2018, true);
draft2017.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2017.Rows.Count()));
draft2017["DraftYear"].FillNulls(2017, true);
draft2017["To"].FillNulls(2017, true);
draft2016.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2016.Rows.Count()));
draft2016["DraftYear"].FillNulls(2016, true);
draft2016["To"].FillNulls(2016, true);
draft2015.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2015.Rows.Count()));
draft2015["DraftYear"].FillNulls(2015, true);
draft2015["To"].FillNulls(2015, true);

OK, let's get the median age of a drafted player, then insert that age whenever we don't have that data.

In [ ]:
medianAge = draft2021["Age"].Median();
int age = (int)medianAge;
draft2021["Age"].FillNulls(age, true);
draft2020["Age"].FillNulls(age, true);
draft2019["Age"].FillNulls(age, true);
draft2018["Age"].FillNulls(age, true);
draft2017["Age"].FillNulls(age, true);
draft2016["Age"].FillNulls(age, true);
draft2015["Age"].FillNulls(age, true);

All other null values can be zero.

In [ ]:
draft2021["Pos"].FillNulls("XX", true);

// Possible null rows are: wAV,DrAV,G,PassCmp,PassAtt,PassYds,PassTD,PassInt,RushAtt,RushYds,RushTD,Rec,RecYds,RecTD,SoloTak,Int,Sk
draft2021["wAV"].FillNulls(0, true);
draft2021["DrAV"].FillNulls(0, true);
draft2021["G"].FillNulls(0, true);
draft2021["PassCmp"].FillNulls(0, true);
draft2021["PassAtt"].FillNulls(0, true);
draft2021["PassYds"].FillNulls(0, true);
draft2021["PassTD"].FillNulls(0, true);
draft2021["PassInt"].FillNulls(0, true);
draft2021["RushAtt"].FillNulls(0, true);
draft2021["RushYds"].FillNulls(0, true);
draft2021["RushTD"].FillNulls(0, true);
draft2021["Rec"].FillNulls(0, true);
draft2021["RecYds"].FillNulls(0, true);
draft2021["RecTD"].FillNulls(0, true);
draft2021["SoloTak"].FillNulls(0, true);
draft2021["Int"].FillNulls(0, true);
draft2021["Sk"].FillNulls("0", true);

Microsoft.Data.Analysis.WriteCSV(draft2021,"newCSV.csv",",");

draft2021.ToTabularDataResource()
//draft2021.ToTabularDataResource().ExploreWithSandDance().Display()